In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122756")

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122756
Azure region: southcentralus
Subscription id: 502bce8d-d66b-4661-a423-f21e61d20123
Resource group: aml-quickstarts-122756


In [16]:
from azureml.core.compute import ComputeTarget, AmlCompute
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
comp_cluster = 'opt-ml-cluster'
comp_confg = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4, min_nodes=1)
comp_trget = ComputeTarget.create(ws, comp_cluster, comp_confg)

In [17]:
comp_trgets = ws.compute_targets
for comp_name,comp_type in comp_trgets.items():
    print(comp_name,comp_type.type,comp_type.provisioning_state)

project1 ComputeInstance Succeeded
opt-ml-cluster AmlCompute Succeeded


In [18]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--batch-size' : choice(25,50,75,100),
    '--learning-rate' : uniform(0.07,1)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory = '.',
    compute_target = comp_trget,
    entry_script = 'train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=5,
    policy=policy,
    estimator=est
)

In [21]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
hyperdrive_submission = exp.submit(config=hyperdrive_config)
hyperdrive_submission.wait_for_completion(show_output=True)

RunId: HD_402d014f-07d3-4c64-b158-86d3d0890819
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_402d014f-07d3-4c64-b158-86d3d0890819?wsid=/subscriptions/502bce8d-d66b-4661-a423-f21e61d20123/resourcegroups/aml-quickstarts-122756/workspaces/quick-starts-ws-122756

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-10-25T05:18:02.712826][API][INFO]Experiment created<END>\n""<START>[2020-10-25T05:18:03.791644][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2020-10-25T05:18:04.279677][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-10-25T05:18:05.0309346Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_402d014f-07d3-4c64-b158-86d3d0890819
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_402d014f-07d3-4c64-b158-86d3d0890819?wsid=/subscriptions/502bc

{'runId': 'HD_402d014f-07d3-4c64-b158-86d3d0890819',
 'target': 'opt-ml-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2020-10-25T05:18:01.922729Z',
 'endTimeUtc': '2020-10-25T05:22:35.704274Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/502bce8d-d66b-4661-a423-f21e61d20123/resourceGroups/aml-quickstarts-122756/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-122756/experiments/**SCRUBBED**/runs/HD_402d014f-07d3-4c64-b158-86d3d0890819_0, {\'error\': {\n    "additional_properties": {\n        "componentName": null\n    },\n    "error": {\n        "additional_properties": {\n            "referenceCode": null,\n            "messageFormat": null,\n            "severity": null,\n            "messageParameters": {}\n        },\n        "code": "UserError",\n        "message": "User pro

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ',best_run.id)
best_metric = best_run.get_metrics()
print('Accuracy : ',best_metric['accuracy'])

final_model = best_run.register_model(model_name = 'hyper-logistic-model',model_path = '/outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_link = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=,
    label_column_name='outcome',
    n_cross_validations=7)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
class_automl = exp.submit(automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
class_automl, best_fit_model = class_automl.get_output()
print(best_fit_model)

joblib.dump(best_fit_model, 'outputs/best_fit_automl_model.joblib')